In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import fetch_openml

In [3]:
mnist = fetch_openml('mnist_784', as_frame=False)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
X_train, y_train = mnist.data, mnist.target

In [5]:
X_train.shape

(70000, 784)

In [6]:
y_train.shape

(70000,)

In [7]:
X_train, X_test, y_train, y_test = X_train[:60000], X_train[60000:], y_train[:60000], y_train[60000:]

In [8]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(60000, 784) (10000, 784) (60000,) (10000,)


In [9]:
from sklearn.svm import SVC

In [10]:
svc_clf = SVC(random_state=42)

In [12]:
svc_clf.fit(X_train[:2000], y_train[:2000])

SVC(random_state=42)

In [13]:
y_train[15]

'7'

In [28]:
test_digit = [X_train[i] for i in range(15)]

In [33]:
for i in range(15):
  print(svc_clf.predict([test_digit[i]]))

['5']
['0']
['4']
['1']
['9']
['2']
['1']
['3']
['1']
['4']
['3']
['5']
['3']
['6']
['1']


In [37]:
y_train[:15]

array(['5', '0', '4', '1', '9', '2', '1', '3', '1', '4', '3', '5', '3',
       '6', '1'], dtype=object)

In [38]:
from sklearn.model_selection import cross_val_score

In [39]:
cross_val_score(svc_clf, X_train[:2000], y_train[:2000], cv=3, scoring="accuracy")

array([0.91754123, 0.89955022, 0.93693694])

In [40]:
from sklearn.multiclass import OneVsRestClassifier

In [42]:
ovr_clf = OneVsRestClassifier(SVC(random_state=42))

In [43]:
ovr_clf.fit(X_train[:2000], y_train[:2000])

OneVsRestClassifier(estimator=SVC(random_state=42))

In [44]:
some_digit = X_train[17]

In [46]:
ovr_clf.predict([some_digit])

array(['8'], dtype='<U1')

In [45]:
y_train[17]

'8'

In [47]:
cross_val_score(ovr_clf, X_train[:2000], y_train[:2000], cv=3, scoring="accuracy")

array([0.91454273, 0.88455772, 0.93693694])

In [48]:
from sklearn.linear_model import SGDClassifier

In [52]:
sgd_clf = SGDClassifier(random_state=42)

In [53]:
sgd_clf.fit(X_train, y_train)

SGDClassifier(random_state=42)

In [54]:
test_digit = X_train[34]

In [55]:
sgd_clf.predict([test_digit])

array(['0'], dtype='<U1')

In [56]:
y_train[34]

'0'

In [57]:
cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring="accuracy")

array([0.87365, 0.85835, 0.8689 ])

In [58]:
from sklearn.neighbors import KNeighborsClassifier

In [71]:
knn_clf = KNeighborsClassifier()

In [72]:
knn_clf.fit(X_train, y_train)

KNeighborsClassifier()

In [73]:
test_digit = X_train[64]

In [74]:
knn_clf.predict([test_digit])

array(['4'], dtype=object)

In [75]:
y_train[64]

'4'

In [76]:
cross_val_score(knn_clf, X_train, y_train, cv=3, scoring="accuracy")

array([0.9676 , 0.9671 , 0.96755])

In [77]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [78]:
pipeline = Pipeline([
    ("kneighbours", KNeighborsClassifier())
])

In [79]:
param_grid = [
    {
        'kneighbours__n_neighbors': [2, 4, 8],
        'kneighbours__weights': ['uniform']},
    {
        'kneighbours__n_neighbors': [5, 10, 15],
        'kneighbours__weights': ['uniform', 'distance']
    }
]

In [80]:
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring="accuracy")

In [82]:
grid_search.fit(X_train[:2000], y_train[:2000])

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kneighbours',
                                        KNeighborsClassifier())]),
             param_grid=[{'kneighbours__n_neighbors': [2, 4, 8],
                          'kneighbours__weights': ['uniform']},
                         {'kneighbours__n_neighbors': [5, 10, 15],
                          'kneighbours__weights': ['uniform', 'distance']}],
             scoring='accuracy')

In [83]:
grid_search.best_params_

{'kneighbours__n_neighbors': 5, 'kneighbours__weights': 'distance'}

In [84]:
knn_clf = KNeighborsClassifier(n_neighbors=5, weights='distance')

In [85]:
knn_clf.fit(X_train, y_train)

KNeighborsClassifier(weights='distance')

In [87]:
test_digit = X_train[103]

In [88]:
knn_clf.predict([test_digit])

array(['7'], dtype=object)

In [89]:
y_train[103]

'7'

In [90]:
cross_val_score(knn_clf, X_train, y_train, cv=3, scoring="accuracy")

array([0.9688 , 0.96795, 0.96905])

In [91]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [94]:
param_distribs = [
    {
        'kneighbours__n_neighbors': randint(low=3, high=20),
        'kneighbours__weights': ['uniform', 'distance']
    }
]

In [95]:
rnd_search = RandomizedSearchCV(pipeline, param_distributions=param_distribs, n_iter=10, cv=3, scoring="accuracy", random_state=42)

In [97]:
rnd_search.fit(X_train[:2000], y_train[:2000])

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('kneighbours',
                                              KNeighborsClassifier())]),
                   param_distributions=[{'kneighbours__n_neighbors': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79a5fd7e3520>,
                                         'kneighbours__weights': ['uniform',
                                                                  'distance']}],
                   random_state=42, scoring='accuracy')

In [98]:
rnd_search.best_params_

{'kneighbours__n_neighbors': 4, 'kneighbours__weights': 'distance'}

In [99]:
knn_clf = KNeighborsClassifier(n_neighbors=4, weights='distance')

In [100]:
knn_clf.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=4, weights='distance')

In [101]:
test_digit = X_train[42]

In [103]:
knn_clf.predict([test_digit])

array(['7'], dtype=object)

In [104]:
y_train[42]

'7'

In [105]:
cross_val_score(knn_clf, X_train, y_train, cv=3, scoring="accuracy")

array([0.9709 , 0.9698 , 0.97035])